In [14]:
!pip install RegNet

In [15]:
import tensorflow.compat.v2 as tf

from keras import backend
from keras import layers
from keras.applications import imagenet_utils
from keras.engine import training
from keras.utils import data_utils
from keras.utils import layer_utils

# isort: off
from tensorflow.python.util.tf_export import keras_export

BASE_WEIGHTS_PATH = (
    "/content/vgg16.h5"
    #"https://storage.googleapis.com/tensorflow/keras-applications/regnet/"
)

WEIGHTS_HASHES = {
    "x002": (
        "49fb46e56cde07fdaf57bffd851461a86548f6a3a4baef234dd37290b826c0b8",
        "5445b66cd50445eb7ecab094c1e78d4d3d29375439d1a7798861c4af15ffff21",
    ),
}

In [16]:

MODEL_CONFIGS = {
    "x002": {
        "depths": [1, 1, 4, 7],
        "widths": [24, 56, 152, 368],
        "group_width": 8,
        "default_size": 224,
        "block_type": "X",
    },
}

In [17]:
@keras_export(
    "keras.applications.regnet.RegNetX002", "keras.applications.RegNetX002"
)
def RegNetX002(
    model_name="regnetx002",
    include_top=True,
    include_preprocessing=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
):
    return RegNet(
        MODEL_CONFIGS["x002"]["depths"],
        MODEL_CONFIGS["x002"]["widths"],
        MODEL_CONFIGS["x002"]["group_width"],
        MODEL_CONFIGS["x002"]["block_type"],
        MODEL_CONFIGS["x002"]["default_size"],
        model_name=model_name,
        include_top=include_top,
        include_preprocessing=include_preprocessing,
        weights=weights,
        input_tensor=input_tensor,
        input_shape=input_shape,
        pooling=pooling,
        classes=classes,
        classifier_activation=classifier_activation,
    )


In [19]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import RegNetX002
from tensorflow.keras.applications import regnet


# Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

# # Define data paths
# train_data_dir = '/content/drive/MyDrive/dissertation/TB Chest Radiography Database/Cleaned Data/Tuberculosis'
# val_data_dir = '/content/drive/MyDrive/dissertation/TB Chest Radiography Database/Cleaned Data/Normal'
DATA_PATH = "/kaggle/input/tuberculosis-tb-chest-xray-dataset/TB_Chest_Radiography_Database/"
files = os.listdir(DATA_PATH)
print(files)
# Define image size and other parameters
img_size = (224, 224)
batch_size = 32
num_classes = 2

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=10,
    validation_split=0.2
)

print('Train Set Generated - ',end=' ') 
train_generator = train_datagen.flow_from_directory(
    DATA_PATH,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
   # classes=['normal', 'tuberculosis'],
    color_mode='rgb',
    subset='training'
)

print('Validation Set Generated - ',end=' ') 
val_generator = train_datagen.flow_from_directory(
    DATA_PATH,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
   # classes=['normal', 'tuberculosis'],
    color_mode='rgb',
    subset='validation'
)

# Create RegNetX002 model
base_model = regnet.RegNetX002(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)

# Add a new classifier on top
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

model = tf.keras.models.Model(inputs=base_model.input, outputs=output)

# Freeze the base model's layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10
steps_per_epoch = train_generator.n // batch_size
validation_steps = val_generator.n // batch_size

model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=validation_steps
)

# Save the trained model
model.save('/content/drive/MyDrive/TB_Model/regnetx002_model.h5')


['Normal.metadata.xlsx', 'Tuberculosis', 'README.md.txt', 'Tuberculosis.metadata.xlsx', 'Normal']
Train Set Generated -  Found 3360 images belonging to 2 classes.
Validation Set Generated -  Found 840 images belonging to 2 classes.
Epoch 1/10
105/105 [==============================] - 112s 959ms/step - loss: 0.4922 - accuracy: 0.8247 - val_loss: 0.4505 - val_accuracy: 0.8341
Epoch 2/10
105/105 [==============================] - 76s 724ms/step - loss: 0.4666 - accuracy: 0.8333 - val_loss: 0.4542 - val_accuracy: 0.8317
Epoch 3/10
105/105 [==============================] - 81s 768ms/step - loss: 0.4648 - accuracy: 0.8333 - val_loss: 0.4531 - val_accuracy: 0.8317
Epoch 4/10
105/105 [==============================] - 81s 770ms/step - loss: 0.4618 - accuracy: 0.8333 - val_loss: 0.4498 - val_accuracy: 0.8341
Epoch 5/10
105/105 [==============================] - 75s 717ms/step - loss: 0.4599 - accuracy: 0.8333 - val_loss: 0.4496 - val_accuracy: 0.8341
Epoch 6/10
105/105 [======================

In [21]:
test_loss, test_accuracy = model.evaluate(train_generator, steps=len(train_generator))
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


105/105 [==============================] - 59s 565ms/step - loss: 0.4558 - accuracy: 0.8333
Test Loss: 0.45580700039863586
Test Accuracy: 0.8333333134651184


In [22]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 regnetx002_prestem_rescaling (  (None, 224, 224, 3)  0          ['input_6[0][0]']                
 Rescaling)                                                                                       
                                                                                                  
 regnetx002_stem_conv (Conv2D)  (None, 112, 112, 32  864         ['regnetx002_prestem_rescaling[0]
                                )                                [0]']                      